# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [18]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [19]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [20]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data = pd.read_csv(Path("Data/sfo_neighborhoods_census_data.csv"), index_col="year")

neighborhood_locations = pd.read_csv(Path("Data/neighborhoods_coordinates.csv"))
neighborhood_locations.columns = ["neighborhood", "Lat", "Lon"]
print(sfo_data.head())
print(neighborhood_locations.head())

          neighborhood  sale_price_sqr_foot  housing_units  gross_rent
year                                                                  
2010      Alamo Square           291.182945         372560        1239
2010        Anza Vista           267.932583         372560        1239
2010           Bayview           170.098665         372560        1239
2010  Buena Vista Park           347.394919         372560        1239
2010  Central Richmond           319.027623         372560        1239
      neighborhood        Lat         Lon
0     Alamo Square  37.791012 -122.402100
1       Anza Vista  37.779598 -122.443451
2          Bayview  37.734670 -122.401060
3  Bayview Heights  37.728740 -122.410980
4   Bernal Heights  37.728630 -122.443050


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [21]:
# caculations reused for different functions below 
top_ten_expensive_neighborhood=sfo_data.groupby(['neighborhood']).mean().sort_values(by=['sale_price_sqr_foot'],ascending=False).reset_index().head(10)
neighborhood_mean=sfo_data.groupby(['neighborhood']).mean().sort_values(by=['sale_price_sqr_foot'],ascending=False).reset_index()
combined_df=pd.concat([neighborhood_mean.set_index('neighborhood'),neighborhood_locations.set_index('neighborhood')],axis='columns',join='inner').reset_index()
df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(top_ten_expensive_neighborhood["neighborhood"])].reset_index()
df_expensive_neighborhoods=sfo_data.groupby(['neighborhood']).mean().sort_values(by=['sale_price_sqr_foot'],ascending=False).reset_index().head(10)


In [22]:

# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units = sfo_data.groupby('year').mean()
    
    min = housing_units.min()['housing_units']
    max = housing_units.max()['housing_units']

    plot=housing_units.hvplot.bar(x='year',y='housing_units',ylim =(min-2000, max+2000),title="Average Housing Units/Year in San Francisco", figsize=(6,4),yformatter='%.0f')
    return plot


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    avg_sfo_data_df=sfo_data.groupby(sfo_data.index)['sale_price_sqr_foot','gross_rent'].mean()
    return avg_sfo_data_df['gross_rent'].hvplot( figsize=(6,4))

def average_sales_price():
    """Average Sales Price Per Year."""
    avg_sfo_data_df=sfo_data.groupby(sfo_data.index)['sale_price_sqr_foot','gross_rent'].mean()
    return avg_sfo_data_df['sale_price_sqr_foot'].hvplot( figsize=(6,4))

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    mean_by_year_and_neighborhood=sfo_data.groupby([sfo_data.index,'neighborhood']).mean().reset_index()
    mean_by_year_and_neighborhood.head()
    return mean_by_year_and_neighborhood.hvplot(x='year',y='sale_price_sqr_foot',groupby='neighborhood')

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_ten_expensive_neighborhood=sfo_data.groupby(['neighborhood']).mean().sort_values(by=['sale_price_sqr_foot'],ascending=False).reset_index().head(10)
    return top_ten_expensive_neighborhood.hvplot.bar(y='sale_price_sqr_foot',x='neighborhood',ylabel='Avg Sale Price per Square Foot',groupby='neighborhood')


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    mean_by_year_and_neighborhood=sfo_data.groupby([sfo_data.index,'neighborhood']).mean().reset_index()
    return mean_by_year_and_neighborhood.hvplot.bar("year", ["gross_rent", "sale_price_sqr_foot"],groupby= "neighborhood",rot=90)
    

    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    return px.parallel_categories(
            df_expensive_neighborhoods,
            dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],
            color="sale_price_sqr_foot",
            color_continuous_scale=px.colors.sequential.Inferno,
        )



def parallel_categories():
    """Parallel Categories Plot."""
    return px.parallel_coordinates(df_expensive_neighborhoods,dimensions=['sale_price_sqr_foot','housing_units','gross_rent'] ,color='sale_price_sqr_foot')



def neighborhood_map():
    """Neighborhood Map."""

    mapbox_token=os.environ['mapbox']
    # Create a scatter mapbox to analyze neighborhood info
    px.set_mapbox_access_token(map_box_api)

    plot= px.scatter_mapbox(
                combined_df,
                lat="Lat",
                lon="Lon",
                size="sale_price_sqr_foot",
                color="neighborhood",
                zoom=10
            )
    return plot


def sunburst():
    """Sunburst Plot."""
    
    return px.sunburst(df_expensive_neighborhoods_per_year, path=['year', 'neighborhood'], values='sale_price_sqr_foot',color='gross_rent',color_continuous_scale="blues", title = "Cost Analysis of Most Expensive Neighborhoods in San Francisco per Year")



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [23]:


title = '##Real Estate Analysis of San Francisco'
welcome_tab = pn.Column(pn.Column(title), neighborhood_map())

market_analysis_row = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())


neighborhood_analysis_tab = pn.Column(average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_plots_tab = pn.Column( 
    parallel_categories(),
    parallel_coordinates()
)

parallel_plots_tab = pn.Column( 
    parallel_categories(),
    parallel_coordinates()
)

# Create tabs

all_tabs = pn.Tabs(("Welcome", welcome_tab), 
("Yearly Market Analysis", market_analysis_row),
("Neighborhood Analysis", neighborhood_analysis_tab), 
("Parallel Plot Analysis", parallel_plots_tab),
("Sunburst Plot Analysis", sunburst())
)


# Create the dashboard
SF_dashboard= pn.Column( 
    "#Real Estate Analysis of SFO from 2010 to 2016",
    all_tabs
)

/Users/manishrajkarnikar/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/Users/manishrajkarnikar/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



## Serve the Panel Dashboard

In [24]:
# Serve the# dashboard
SF_dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'78799880-c0bc-493e-89c9-361b90c6d023': {'defs': …

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Column
                [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Union Square District', ...], value='Union Square District', width=250)
                    [1] VSpacer()
        [3] Column
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [4] Plotly(Figure, name='Sunburst Plot Analysis')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [25]:
housing_units_per_year()

:Bars   [year]   (housing_units)

In [26]:
average_gross_rent()

/Users/manishrajkarnikar/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



:Curve   [year]   (gross_rent)

In [27]:
average_sales_price()

/Users/manishrajkarnikar/opt/anaconda3/envs/alpacaenv/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



:Curve   [year]   (sale_price_sqr_foot)

In [28]:
average_price_by_neighborhood()

BokehModel(combine_events=True, render_bundle={'docs_json': {'d8aead04-aa3f-4ff7-9253-5b0585b6aa25': {'defs': …

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot)

In [29]:
top_most_expensive_neighborhoods()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0097619a-e699-482a-afe2-891838077c10': {'defs': …

:DynamicMap   [neighborhood]
   :Bars   [neighborhood]   (sale_price_sqr_foot)

In [30]:
most_expensive_neighborhoods_rent_sales()

BokehModel(combine_events=True, render_bundle={'docs_json': {'868a7f04-3254-4e7d-a723-3d0bb4dd8d1a': {'defs': …

:DynamicMap   [neighborhood]
   :Bars   [year,Variable]   (value)

In [31]:
neighborhood_map()

In [32]:
parallel_categories()

In [33]:
parallel_coordinates()

In [34]:
sunburst()